# LangGraph 进阶概念：中间件 (Middleware) & 人机交互 (Human-in-the-Loop)

欢迎来到 LangGraph 进阶课程！本 Notebook 建立在 LangGraph 101 的基础之上，介绍构建生产级 Agent 的两个强大模式。

**你将学到：**
- **Human-in-the-Loop (人机交互)** - 暂停 Agent 以进行人工审查和批准
- **Middleware (中间件)** - 在执行的关键点修改 Agent 的行为
- **Tool Review (工具审查)** - 为敏感工具添加审批工作流
- **Dynamic Behavior (动态行为)** - 根据上下文调整 Agent 的响应

**先决条件：** 完成 `langgraph_101.ipynb`
</br>
</br>

---
</br>

> **注意：** 这些模式对于生产级 Agent 至关重要，因为在生产环境中，安全性、合规性和用户控制是必不可少的。LangGraph 1.0 引入了 `interrupt` 和 `Command` 等新原语，让这些模式的实现变得更加直观和强大。

### LangGraph 1.0 升级指南 - 人机交互与控制流

在 LangGraph 1.0 中，**Human-in-the-Loop** 和 **控制流** 发生了重大变化，变得更加动态和灵活。

**主要区别：**

#### 1. 中断机制：`interrupt()` vs 静态配置
- **旧版 (0.x)**: 需要在编译 Graph 时通过 `interrupt_before=["node_name"]` 静态指定在哪停。
- **新版 (1.0)**: 引入了动态 **`interrupt()`** 函数。你可以在任何节点或工具内部调用它。这允许基于逻辑（如“置信度低时才中断”）动态决定是否暂停。

#### 2. 状态更新与导航：`Command` 对象
- **旧版 (0.x)**: 节点返回状态更新，路由由单独的条件边函数处理。
- **新版 (1.0)**: 引入 **`Command`** 对象。一个节点可以同时返回状态更新 (`update`) 和路由指令 (`goto`)，极大简化了复杂逻辑的编写。

本教程的“第 1 部分”和“第 2 部分”将详细演示这些新特性的用法。

## 0.1 环境配置与工具整合 (Environment & Utils)

为了使本 Notebook 自包含，我们将原本分散在 `utils` 目录和 `.env` 文件的配置整合到这里。
The following cells merged from `utils/models.py`, `utils/utils.py`, and `.env.example`.

In [ ]:
# Configuration: Set Environment Variables (derived from .env.example)
# 配置: 设置环境变量 (源自 .env.example)
import os

# If using OpenAI 

# os.environ['OPENAI_API_KEY'] = "<openai-api-key>"
# os.environ['OPENAI_BASE_URL'] = "<openai-base-url>" # Optional, if different from the default"

# Optional for LangSmith tracing and experiment tracking

# os.environ['LANGSMITH_API_KEY'] = "<langsmith-api-key>"
# os.environ['LANGSMITH_ENDPOINT'] = "<langsmith-endpoint>" #defaults to https://api.smith.langchain.com if using cloud"
# os.environ['LANGSMITH_TRACING'] = "true"
# os.environ['LANGSMITH_PROJECT'] = "langgraph-102"

# If using Azure OpenAI

# os.environ['AZURE_OPENAI_API_KEY'] = "<azure-openai-api-key>"
# os.environ['AZURE_OPENAI_ENDPOINT'] = "<azure-openai-endpoint>"
# os.environ['AZURE_OPENAI_API_VERSION'] = ""

# If using Anthropic

# os.environ['ANTHROPIC_API_KEY'] = "<anthropic-api-key>"

# If using AWS 

# os.environ['AWS_ACCESS_KEY_ID'] = ""
# os.environ['AWS_SECRET_ACCESS_KEY'] = ""
# os.environ['AWS_REGION_NAME'] = ""
# os.environ['AWS_MODEL_ARN'] = ""

# if using Google Vertex AI
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./vertexCred.json" # replace with path to your vertex credentials"


In [ ]:
# Content from utils/utils.py
import sqlite3
import requests
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

def show_graph(graph, xray=False):
    """展示 LangGraph 的 Mermaid 流程图，如果失败则使用 ASCII 图表作为后备。
    
    参数:
        graph: 拥有 get_graph() 方法的 LangGraph 对象
        xray: 是否展示图的内部结构 (例如子图)
    """
    from IPython.display import Image
    try:
        return Image(graph.get_graph(xray=xray).draw_mermaid_png())
    except Exception as e:
        print(f"⚠️  图片渲染失败: {e}")
        print("\n📊 改为展示 ASCII 图表:\n")
        ascii_diagram = graph.get_graph(xray=xray).draw_ascii()
        print(ascii_diagram)
        return None

def get_engine_for_chinook_db():
    """下载 SQL 文件，填充内存数据库，并创建 SQLAlchemy 引擎。"""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


In [ ]:
# Content from utils/models.py
"""
模型初始化文件

此文件配置应用程序中使用的 LLM (大语言模型)。

默认配置:
- 默认提供商是 OpenAI (使用 o3-mini 模型)
- 你也可以通过取消注释相应的行来切换到 Anthropic

备选提供商:
要使用不同的 LLM 提供商，请按照以下步骤操作:
1. 注释掉下方的 "Default Models" (默认模型) 部分
2. 取消注释你想要的提供商部分:
   - Azure OpenAI: 需要设置 AZURE_OPENAI_API_KEY 和 AZURE_OPENAI_ENDPOINT
   - AWS Bedrock: 需要设置 AWS 凭证和配置
   - Google Vertex AI: 需要设置 GOOGLE_APPLICATION_CREDENTIALS
3. 按照每个部分内的设置说明进行操作
"""

"""默认模型 (Default Models)"""
from dotenv import load_dotenv
# 加载 .env 环境变量，覆盖现有变量
load_dotenv(dotenv_path="../../.env", override=True)
from langchain.chat_models import init_chat_model

# 初始化聊天模型，这里使用 OpenAI 的 o3-mini
model = init_chat_model("openai:o3-mini")

# 使用 Anthropic 替代 OpenAI
# model = init_chat_model("anthropic:claude-haiku-4-5")

# ... (rest of the file logic is commented out, keeping it simple)


## 设置

让我们快速设置环境。

In [ ]:
# 将项目根目录添加到 Python 路径
import sys
from pathlib import Path
project_root = Path().resolve().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 导入模型配置
# from utils.models import model # Copied inline above

# 备选方案：内联定义模型
# from dotenv import load_dotenv
# load_dotenv(dotenv_path="../.env", override=True)
# from langchain.chat_models import init_chat_model
# model = init_chat_model("openai:o3-mini")

## 第 1 部分: 使用 Interrupts 实现人机交互 (Human-in-the-Loop)

### 问题背景

想象一下，你正在构建一个可以发送电子邮件或进行购买的 Agent。你肯定不希望它自动执行这些操作 —— 你希望先经过人工批准！

**Human-in-the-Loop** 允许你：
- 暂停执行以进行审查
- 批准、拒绝或编辑动作
- 为敏感操作添加安全控制

### 工作原理 (LangGraph 1.0 新特性)

LangGraph 1.0 引入了动态 `interrupt` 函数，替代了旧版较为静态的配置方式。

1. Agent 遇到 `interrupt()` 函数 - 执行暂停
2. 系统将信息展示给人类
3. 人类提供输入（批准/拒绝/编辑）
4. Agent 使用 `Command(resume=...)` 恢复执行，并将人类输入作为 `interrupt()` 的返回值传回

### 示例 1: 简单的审批工作流

让我们从一个简单的例子开始——在发送电子邮件之前请求批准。

In [ ]:
from langgraph.types import interrupt
from langchain_core.tools import tool

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """发送邮件给接收者。"""
    
    # 暂停以等待人工批准
    # interrupt 接收的参数会作为暂停时的信息返回给前端/用户
    # 程序会在这里挂起，直到收到 resume 指令
    approval = interrupt({
        "action": "send_email",
        "to": to,
        "subject": subject,
        "body": body,
        "message": "Do you want to send this email?"
    })
    
    # 恢复执行后，approval 变量将包含 resume 时传入的数据
    if approval.get("approved"): # 如果通过批准（根据我们定义的协议）
        # 在生产环境中，这里会实际发送邮件
        return f" Email sent to {to} with subject '{subject}'"
    else:
        return "Email cancelled by user"

# 测试工具定义
print("Tool created successfully!")
print(f"Tool name: {send_email.name}")
print(f"Tool description: {send_email.description}")

### 创建带有 Human-in-the-Loop 的 Agent

现在让我们创建一个使用此工具的 Agent。**切记：** Interrupts 必须配合 Checkpointer 使用，否则无法保存暂停时的状态！

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver

# 创建 Checkpointer 用于持久化
checkpointer = MemorySaver()

# 创建带有邮件工具的 Agent
agent = create_agent(
    model=model,
    tools=[send_email],
    system_prompt="You are a helpful email assistant. When asked to send emails, use the send_email tool.",
    checkpointer=checkpointer  # 必须项：用于 interrupts
)

### 运行直到中断 (Running Until Interrupt)

让我们运行 Agent，看看它如何暂停以等待批准：

In [ ]:
from langchain.messages import HumanMessage
from langsmith import uuid7

# 为此对话创建唯一线程
config = {"configurable": {"thread_id": uuid7()}}

# 运行 Agent
result = agent.invoke(
    {
        "messages": [HumanMessage(content="Send an email to alice@example.com with subject 'Meeting Tomorrow' and body 'Let's meet at 3pm.'")]
    },
    config=config
)

# 检查是否触发了 interrupt
# 在 LangGraph 1.0 中，如果因 interrupt 暂停，结果中会包含 __interrupt__ 键
if "__interrupt__" in result:
    print("Agent paused for approval (Agent 已暂停等待批准)\n")

    # 获取 interrupt 的详细信息
    interrupt_info = result["__interrupt__"][0]

    print("Interrupt details:")
    print(f"  To: {interrupt_info.value['to']}")
    print(f"  Subject: {interrupt_info.value['subject']}")
    print(f"  Body: {interrupt_info.value['body']}")
    print(f"  Message: {interrupt_info.value['message']}")
else:
    print("Agent completed without interrupt")

### 批准并恢复执行 (Resuming with Approval)

现在让我们批准这封邮件，并让 Agent 继续运行。
我们使用 `Command` 对象并传入 `resume` 参数。这个值会直接成为 `interrupt()` 函数在工具内部的返回值。

In [ ]:
from langgraph.types import Command

# 恢复执行并给予批准
# 这里的 {"approved": True} 将会是工具中 approval 变量的值
result = agent.invoke(
    Command(resume={"approved": True}),
    config=config
)

# 打印最终响应
print("Final response:")
print(result["messages"][-1].content)

### 练习：尝试拒绝邮件

再次运行上面的步骤，但这次通过传入 `{"approved": False}` 来拒绝邮件：

In [ ]:
# 为拒绝示例创建一个新线程
config_2 = {"configurable": {"thread_id": uuid7()}}

# 运行直到中断
result = agent.invoke(
    {
        "messages": [HumanMessage(content="Send an email to bob@example.com saying 'Hello!'")]
    },
    config=config_2
)

# 恢复并拒绝
result = agent.invoke(
    Command(resume={"approved": False}),  # 拒绝邮件
    config=config_2
)

print("Final response:")
print(result["messages"][-1].content)

## 第 2 部分: 进阶模式 - 执行前先编辑 (Edit Before Execution)

有时你不仅仅想批准或拒绝，还想**编辑**工具调用的参数。让我们升级我们的工具：

In [ ]:
@tool
def send_email_v2(to: str, subject: str, body: str) -> str:
    """发送邮件给接收者。"""
    
    # 暂停等待人工审查
    response = interrupt({
        "action": "send_email",
        "to": to,
        "subject": subject,
        "body": body,
        "message": "Review this email. You can approve, reject, or edit it."
    })
    
    # 处理不同的响应类型
    if response["type"] == "approve":
        return f"Email sent to {to} with subject '{subject}'"

    elif response["type"] == "reject":
        return "Email cancelled"

    elif response["type"] == "edit":
        # 更新为编辑后的值
        to = response.get("to", to)
        subject = response.get("subject", subject)
        body = response.get("body", body)
        return f"""Email sent with edits:
                To: {to}
                Subject: {subject}
                Body: {body}"""
    
    return "Unknown response"

# 创建使用增强工具的新 Agent
agent_v2 = create_agent(
    model=model,
    tools=[send_email_v2],
    system_prompt="You are a helpful email assistant.",
    checkpointer=MemorySaver()
)

In [ ]:
# 运行并编辑邮件
config_3 = {"configurable": {"thread_id": uuid7()}}

# 运行直到中断
result = agent_v2.invoke(
    {
        "messages": [HumanMessage(content="Send an email to team@example.com about the meeting")]
    },
    config=config_3
)

print("Paused for review... (暂停等待审查)\n")

现在，让我们修改邮件主题，将其标记为 URGENT (紧急) 会议！

In [ ]:
# 恢复并带入编辑后的内容
result = agent_v2.invoke(
    Command(resume={
        "type": "edit",
        "subject": "URGENT: Meeting Today at 2pm",  # 我们修改了邮件主题
        "body": "This is the edited email body with more details."
    }),
    config=config_3
)

print("Final response:")
print(result["messages"][-1].content)

## 第 3 部分: 中间件 (Middleware) 简介

**Middleware (中间件)** 提供了对 Agent 循环 (Loop) 的细粒度控制。它可以让你：
- 在模型调用前后检查状态
- 动态修改模型请求
- 在执行的关键点添加自定义逻辑

### Agent 循环 (The Agent Loop)

```
Input --> [before_model] --> [wrap_model_call] --> Model --> [after_model] --> Tools --> ...
```

Middleware 会挂载到这个循环中：
- **`before_model`** - 在模型执行前运行，可以更新状态
- **`wrap_model_call`** - 包装模型调用，控制模型何时/如何被调用（例如重试、缓存）
- **`after_model`** - 在模型执行后，执行工具前运行

### 两种 Hook 风格

**Node-style hooks (节点式钩子)** 按顺序运行：
- `before_agent`, `before_model`, `after_model`, `after_agent`
- 适合用于日志记录、验证、状态更新

**Wrap-style hooks (包装式钩子)** 拦截执行：
- `wrap_model_call`, `wrap_tool_call`
- 完全控制处理程序的调用
- 适合用于重试逻辑、缓存、数据转换

### 示例 1: 动态系统提示词 (Dynamic System Prompt)

让我们创建一个中间件，根据用户的角色动态更改系统提示词：

In [ ]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from typing import TypedDict

# 定义上下文 Schema
class Context(TypedDict):
    user_role: str

# 使用装饰器创建中间件
@dynamic_prompt
def dynamic_prompt_middleware(request: ModelRequest) -> str:
    """根据用户角色调整系统提示词。"""
    
    # 从运行时上下文中获取 user_role
    user_role = request.runtime.context.get("user_role", "general")
    
    if user_role == "expert":
        return "You are an AI assistant for experts. Provide detailed technical responses with code examples."
    elif user_role == "beginner":
        return "You are an AI assistant for beginners. Explain concepts simply, avoid jargon."
    else:
        return "You are a helpful AI assistant."

In [ ]:
from langchain_core.tools import tool

@tool
def explain_concept(concept: str) -> str:
    """Explain a programming concept."""
    explanations = {
        "async": "Asynchronous programming allows code to run without blocking.",
        "recursion": "Recursion is when a function calls itself."
    }
    return explanations.get(concept.lower(), "Concept not found.")

# 创建带有中间件的 Agent
agent_with_middleware = create_agent(
    model=model,
    tools=[explain_concept],
    middleware=[dynamic_prompt_middleware],
    context_schema=Context
)

### 测试不同的用户角色

让我们看看 Agent 如何根据用户角色做出不同的反应：

In [ ]:
# 专家用户
print("=" * 50)
print("EXPERT USER (专家用户)")
print("=" * 50)

result = agent_with_middleware.invoke(
    {"messages": [HumanMessage(content="Explain async programming")]},
    context={"user_role": "expert"}
)
print(result["messages"][-1].content)
print()

# 初学者
print("=" * 50)
print("BEGINNER USER (初学者)")
print("=" * 50)

result = agent_with_middleware.invoke(
    {"messages": [HumanMessage(content="Explain async programming")]},
    context={"user_role": "beginner"}
)
print(result["messages"][-1].content)

### 示例 2: 自定义中间件 - 请求日志记录器 (Request Logger)

中间件允许你 hook 进 Agent 循环，查看每一步发生了什么。这对于调试和理解 Agent 的工作原理非常有用。

**Agent 循环：**
User Input --> [before_model] --> [wrap_model_call] --> Model --> [after_model] --> Tools --> ...

**我们将构建一个日志记录器，在以下步骤打印信息：**
- **Before model** - 当前对话中有多少条消息？
- **Wrap model call** - 正在使用哪个模型和工具？
- **After model** - 模型是调用了工具还是给出了最终答案？

这就像是添加了调试用的 `print()` 语句，但以一种干净、可重用的方式！

In [ ]:
from langchain.agents.middleware import AgentMiddleware, AgentState, ModelRequest, ModelResponse
from typing import Any, Callable

class RequestLoggerMiddleware(AgentMiddleware):
    """用于调试的模型请求日志记录中间件。"""
    
    def before_model(self, state: AgentState, runtime) -> dict[str, Any] | None:
        """在模型执行前记录日志。
        
        Args:
            state: 当前 Agent 的状态 (包含消息历史等)
            runtime: 运行时上下文
        """
        message_count = len(state.get("messages", []))
        print(f"[BEFORE MODEL] Processing {message_count} messages")
        return None  # 不修改状态
    
    def wrap_model_call(
        self, 
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        """包装模型调用，记录请求详情。
        
        Args:
           request: 即将发送给模型的请求对象
           handler: 执行实际模型调用的函数
        """
        print(f"  [MODEL REQUEST]")
        print(f"   Model: {request.model if hasattr(request, 'model') else 'default'}")
        print(f"   Tools available: {len(request.tools) if request.tools else 0}")
        
        # 调用实际的模型处理程序
        return handler(request)
    
    def after_model(self, state: AgentState, runtime) -> dict[str, Any] | None:
        """在模型执行后记录日志。"""
        last_message = state["messages"][-1]
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            print(f" [AFTER MODEL] Model requested {len(last_message.tool_calls)} tool call(s)")
        else:
            print(f" [AFTER MODEL] Model provided final response")
        return None  # 不修改状态

In [ ]:
# 创建带有日志中间件的 Agent
agent_with_logger = create_agent(
    model=model,
    tools=[explain_concept],
    middleware=[RequestLoggerMiddleware()],
)

### 预期结果

当我们运行带日志记录器的 Agent 时，你会实时看到执行流程：

**第一次迭代:**
1. `[BEFORE MODEL]` - 显示我们开始时的消息数量
2. `[MODEL REQUEST]` - 显示可用模型和工具
3. `[AFTER MODEL]` - 模型决定调用 `explain_concept` 工具

**第二次迭代 (工具执行后):**
1. `[BEFORE MODEL]` - 现在消息更多了（包含工具结果）
2. `[MODEL REQUEST]` -再次显示模型信息
3. `[AFTER MODEL]` - 模型给出最终答案（不再需要工具）

这让你能够深入了解 Agent 的决策过程。

让我们运行它：

In [ ]:
# 运行并观察日志
print("\n" + "=" * 50)
print("RUNNING AGENT WITH LOGGER")
print("=" * 50 + "\n")

result = agent_with_logger.invoke({
    "messages": [{"role": "user", "content": "Explain recursion"}]
})

print("\n" + "=" * 50)
print("FINAL RESPONSE")
print("=" * 50)
print(result["messages"][-1].content)

## 第 4 部分: 结合 Middleware 和 Human-in-the-loop

让我们结合 Human-in-the-Loop 和 Middleware，构建一个生产就绪的 Agent：

In [ ]:
# 需要批准的敏感工具
@tool
def delete_database(database_name: str) -> str:
    """Delete a database. THIS IS DANGEROUS!"""
    
    response = interrupt({
        "action": "delete_database",
        "database_name": database_name,
        "warning": "This will permanently delete the database!",
        "message": "Are you absolutely sure?"
    })
    
    if response.get("confirmed"):
        return f"Database '{database_name}' has been deleted (simulation)"
    else:
        return "Database deletion cancelled"

# 用于跟踪危险操作的中间件
class SafetyMiddleware(AgentMiddleware):
    """添加安全检查和日志记录的中间件。"""
    
    name = "safety_checker"
    
    def after_model(self, state: AgentState) -> dict[str, Any] | None:
        """检查危险的工具调用。"""
        last_message = state["messages"][-1]
        
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            for tool_call in last_message.tool_calls:
                if "delete" in tool_call["name"].lower():
                    print("   [SAFETY] Dangerous operation detected!")
                    print(f"   Tool: {tool_call['name']}")
                    print(f"   Args: {tool_call['args']}")
        
        return None

# 创建生产级 Agent
production_agent = create_agent(
    model=model,
    tools=[delete_database],
    middleware=[SafetyMiddleware()],
    checkpointer=MemorySaver()
)

  ### 预期结果：多层安全机制实战

  当我们尝试一个危险操作时，你会看到 **两种** 安全机制被激活：

  **第 1 层 - Middleware 检测:**
  - `[SAFETY] Dangerous operation detected!` - 中间件发现 delete 操作
  - 记录工具名称和参数以用于审计追踪

  **第 2 层 - 人工批准 (Interrupt):**
  - Agent 执行在 `interrupt()` 处暂停
  - 向审核人员展示警告信息
  - 除非获得明确批准，否则执行不会继续

  **这就是纵深防御：** Middleware 监控所有操作，而 Interrupts 强制对关键行动进行人工批准。

In [ ]:
# 测试组合模式
config_4 = {"configurable": {"thread_id": uuid7()}}

print("\n" + "=" * 50)
print("DANGEROUS OPERATION ATTEMPT (危险操作尝试)")
print("=" * 50 + "\n")

# 运行直到 interrupt
result = production_agent.invoke(
    {
        "messages": [HumanMessage(content="Delete the production_db database")]
    },
    config=config_4
)

if "__interrupt__" in result:
    interrupt_info = result["__interrupt__"][0]
    print("\n  Human approval required (需要人工批准):")
    print(f"   {interrupt_info.value['warning']}")
    print(f"   Database: {interrupt_info.value['database_name']}")

print("\n(In a real app, a human would review this before proceeding)")

## 核心要点

### Human-in-the-Loop (Interrupts)
- 使用 `interrupt()` 暂停执行
- 必须配合 `checkpointer` 实现持久化
- 使用 `Command(resume=value)` 恢复执行
- 非常适合审批工作流和敏感操作

### Middleware
- **Node-style hooks**: `before_model`, `after_model` - 顺序逻辑、验证、日志
- **Wrap-style hooks**: `wrap_model_call`, `wrap_tool_call` - 完全控制、重试、转换
- **Decorators**: `@dynamic_prompt`, `@before_model`, `@wrap_model_call` 用于快速定义中间件
- **Classes**: 继承 `AgentMiddleware` 以构建复杂、可重用的组件

### 何时使用什么？

**使用 Interrupts 当：**
- 动作需要人工批准
- 你想审查/编辑工具调用
- 需要验证用户输入

**使用 Middleware 当：**
- 需要动态修改 Agent 行为
- 想要添加日志/监控
- 需要执行策略（Token 限制、安全检查）
- 想要根据上下文个性化响应

**Node-style vs Wrap-style:**
- Node-style 用于顺序操作（日志、验证）
- Wrap-style 用于控制流（重试、回退、缓存）

## 练习 (可选)

尝试构建一个 Agent：
1. 拥有一个进行购买操作的工具
2. 使用中间件检查购买金额是否超过 $1000
3. 如果超过 $1000，使用 interrupt 要求批准
4. 如果低于 $1000，自动处理

提示：结合 `before_model` 中间件和条件性的 `interrupt()` 逻辑！

In [ ]:
# 你的代码！
# 挑战：构建购买审批 Agent

# @tool
# def make_purchase(item: str, amount: float) -> str:
#     ...
#
# class PurchaseLimitMiddleware(AgentMiddleware):
#     ...

## 下一步

你现在已经拥有了构建生产级 Agent 的强大工具！

**继续你的旅程：**
1.  查看 `multi_agent.ipynb` 了解多 Agent 系统
2.  探索内置中间件（摘要 Summarization、Anthropic Prompt Caching）
3.  为你自己的用例构建自定义中间件
4.  添加 LangSmith 进行调试和监控

**资源：**
- [Middleware 文档](https://docs.langchain.com/oss/python/langchain/middleware)
- [Human-in-the-Loop 交互指南](https://docs.langchain.com/oss/python/langchain/human-in-the-loop)
- [LangGraph 文档](https://langchain-ai.github.io/langgraph/)

</br>
</br>
